In [93]:
import os
import sys
import copy
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Slider, Button, CustomJS, Rect, Text, Circle
from bokeh.layouts import column, row
import numpy as np
from bokeh.io import output_file, save
current_dir = "/Users/yoshida.takaya/Documents/ahc/masters2024"

In [105]:
seed = 0

In [106]:
def read_input(seed):
    file_input = os.path.join(current_dir, f"tools/in/{seed:04d}.txt")
    with open(file_input, 'r') as f:
        data = f.read().splitlines()
    
    t, N = map(int, data[0].split())
    
    index = 1
    v = []
    for i in range(N):
        v.append([int(x) for x in data[index]])
        index += 1
    
    h = []
    for i in range(N - 1):
        h.append([int(x) for x in data[index]])
        index += 1
    
    a = []
    for i in range(N):
        a.extend([int(x) for x in data[index].split()])
        index += 1
    
    return t, N, v, h, a

BOARD_TYPE, n, WALL_V, WALL_H, initial_board_num = read_input(seed)

In [107]:
def get_output(seed):
    direction_vectors = {
        "L": (0, -1), "R": (0, 1), "U": (-1, 0), "D": (1, 0), ".": (0, 0)
    }
    file_output_path = os.path.join(current_dir, f"tools/out/{seed:04d}.txt")
    
    with open(file_output_path, 'r') as file:
        lines = file.read().splitlines()
    
    initial_positions = list(map(int, lines[0].split()))
    x1, y1, x2, y2 = initial_positions
    num_moves = len(lines) - 1
    
    swaps = [line.split()[0] == '1' for line in lines[1:]]
    pos_takahashi = [(x1, y1)]
    pos_aoki = [(x2, y2)]
    
    for line in lines[1:]:
        _, move_takahashi, move_aoki = line.split()
        
        last_pos_takahashi = pos_takahashi[-1]
        last_pos_aoki = pos_aoki[-1]
        
        new_pos_takahashi = (
            last_pos_takahashi[0] + direction_vectors[move_takahashi][0],
            last_pos_takahashi[1] + direction_vectors[move_takahashi][1]
        )
        new_pos_aoki = (
            last_pos_aoki[0] + direction_vectors[move_aoki][0],
            last_pos_aoki[1] + direction_vectors[move_aoki][1]
        )
        
        pos_takahashi.append(new_pos_takahashi)
        pos_aoki.append(new_pos_aoki)
    
    return swaps, pos_takahashi, pos_aoki

swaps, pos_takahashi, pos_aoki = get_output(seed)


In [108]:
class Board:
    def __init__(self, n: int, board_num: list[int]):
        self.n = n
        self.board_num = board_num

    def __copy__(self):
        return Board(self.n, self.board_num[:])
    
    def get_board_num(self, i: int, j: int):
        return self.board_num[i*self.n + j]
    
    def get_board_num_2d(self):
        return [self.board_num[i:i+self.n] for i in range(0, len(self.board_num), self.n)]
    
    def change_board_num(self, i: int, j: int, value: int):
        self.board_num[i*self.n + j] = value

    def to_datasource(self) -> ColumnDataSource:
        data = {'x': [], 'y': [], 'width': [], 'height': [], 'color': [], 'text': []}
        min_a = 0
        max_a = self.n*self.n
        for i in range(self.n):
            for j in range(self.n):
                norm_value = (self.board_num[i*self.n + j] - min_a) / (max_a - min_a)
                fill_color = f"rgb({int(255 * norm_value)}, {int(255 * (1 - norm_value))}, 0)"
                data['x'].append(j + 0.5)
                data['y'].append(self.n - i - 0.5)
                data['width'].append(1)
                data['height'].append(1)
                data['color'].append(fill_color)
                data['text'].append(str(self.board_num[i*self.n + j]))
        return ColumnDataSource(data)

In [109]:
class Position:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

    def __copy__(self):
        return Position(self.x, self.y)
    
    def set_position(self, x: int, y: int):
        self.x = x
        self.y = y
    
    def get_position(self):
        return (self.x, self.y)
    
    def to_datasource(self) -> ColumnDataSource:
        data = {'x': [], 'y': []}
        data['x'].append(self.x + 0.5)
        data['y'].append(self.y + 0.5)
        return ColumnDataSource(data)

In [110]:
class Simulator:
    def simulate(self, n: int, initial_board_num: list[int], swaps: list[int], pos_takahashi: list[tuple[int,int]], pos_aoki: list[tuple[int,int]]) -> tuple[list[Board], list[Position], list[Position]]:
        board = Board(n, initial_board_num)
        takahashi_pos = Position(pos_takahashi[0][0], pos_takahashi[0][1])
        aoki_pos = Position(pos_aoki[0][0], pos_aoki[0][1])
        boards = [board]
        takahashi_pos_list = [takahashi_pos]
        aoki_pos_list = [aoki_pos]

        for swap, pos_takahashi, pos_aoki in zip(swaps, pos_takahashi, pos_aoki):
            next_board = copy.copy(board)
            new_takahashi_pos = copy.copy(takahashi_pos)
            new_aoki_pos = copy.copy(aoki_pos)
            if swap:
                v1 = next_board.get_board_num(takahashi_pos.x, takahashi_pos.y)
                v2 = next_board.get_board_num(aoki_pos.x, aoki_pos.y)
                next_board.change_board_num(takahashi_pos.x, takahashi_pos.y, v2)
                next_board.change_board_num(aoki_pos.x, aoki_pos.y, v1)
            new_takahashi_pos.set_position(pos_takahashi[0], pos_takahashi[1])
            new_aoki_pos.set_position(pos_aoki[0], pos_aoki[1])
            boards.append(next_board)
            takahashi_pos_list.append(new_takahashi_pos)
            aoki_pos_list.append(new_aoki_pos)
            board = next_board
            takahashi_pos = new_takahashi_pos
            aoki_pos = new_aoki_pos
        return boards, takahashi_pos_list, aoki_pos_list

In [111]:
boards, takahashi_pos_list, aoki_pos_list = Simulator().simulate(n, initial_board_num, swaps, pos_takahashi, pos_aoki)

In [113]:
class Visualizer:
    def __init__(self, n, wall_v, wall_h):
        # Initialize the Visualizer with grid size and wall information
        self.n = n
        self.wall_v = wall_v
        self.wall_h = wall_h

    def draw_vertical_walls(self, fig: figure) -> figure:
        # Draw vertical walls on the figure
        for i in range(self.n):
            for j in range(self.n - 1):
                if self.wall_v[i][j] == 1:
                    fig.line([j + 1, j + 1], [self.n - i - 1, self.n - i], line_width=4, color='black')
        return fig

    def draw_horizontal_walls(self, fig: figure) -> figure:
        # Draw horizontal walls on the figure
        for i in range(self.n - 1):
            for j in range(self.n):
                if self.wall_h[i][j] == 1:
                    fig.line([j, j + 1], [self.n - i - 1, self.n - i - 1], line_width=4, color='black')
        return fig
    
    def draw_numbers(self, fig: figure, board_datasource: ColumnDataSource) -> None:
        # Draw numbers on the board using rectangles and text
        fig.rect(x='x', y='y', width='width', height='height', color='color', source=board_datasource, alpha=0.4)
        fig.text(x='x', y='y', text='text', source=board_datasource, text_align='center', text_baseline='middle')
    
    def draw_positions(self, fig: figure, takahashi_pos_datasource: ColumnDataSource, aoki_pos_datasource: ColumnDataSource) -> None:
        # Draw positions of Takahashi and Aoki on the board
        fig.circle(x='x', y='y', size=30, color=None, line_color='red', line_width=4, source=takahashi_pos_datasource, name='takahashi')
        fig.circle(x='x', y='y', size=30, color=None, line_color='blue', line_width=4, source=aoki_pos_datasource, name='aoki')

    def _make_frames(self, boards: list[Board], takahashi_pos_list: list[Position], aoki_pos_list: list[Position]):
        # Create frames for each board state and positions
        return [(board.to_datasource(), takahashi_pos.to_datasource(), aoki_pos.to_datasource()) for board, takahashi_pos, aoki_pos in zip(boards, takahashi_pos_list, aoki_pos_list)]

    def _create_slider(self, size):
        # Create a slider for frame navigation
        return Slider(start=0, end=size-1, value=0, step=1, title="Frame")

    def _create_callback(self, frames, rect_source, takahashi_source, aoki_source):
        # Create a JavaScript callback to update data sources based on slider value
        return CustomJS(args=dict(frames=frames, rect_source=rect_source, takahashi_source=takahashi_source, aoki_source=aoki_source), code="""
            const frame_id = cb_obj.value;
            const frame = frames[frame_id];
            rect_source.data = frame[0].data;
            takahashi_source.data = frame[1].data;
            aoki_source.data = frame[2].data;
            rect_source.change.emit();
            takahashi_source.change.emit();
            aoki_source.change.emit();
        """)

    def _create_play_button(self, slider):
        # Create a play button to automatically advance the slider
        play_button = Button(label="Play", button_type="success")
        play_callback = CustomJS(args=dict(slider=slider), code="""
            if (window.playInterval) {
                clearInterval(window.playInterval);
            }
            window.playInterval = setInterval(function() {
                if (slider.value < slider.end) {
                    slider.value += 1;
                } else {
                    clearInterval(window.playInterval);
                }
            }, 20);
        """)
        play_button.js_on_event('button_click', play_callback)
        return play_button

    def _create_pause_button(self):
        # Create a pause button to stop the slider from advancing
        pause_button = Button(label="Pause", button_type="danger")
        pause_callback = CustomJS(code="""
            if (window.playInterval) {
                clearInterval(window.playInterval);
            }
        """)
        pause_button.js_on_event('button_click', pause_callback)
        return pause_button

    def visualize_with_slider(self, boards: list[Board], takahashi_pos_list: list[Position], aoki_pos_list: list[Position]):
        # Visualize the board states with a slider to navigate through frames
        # Make layout of the visualization
        slider = self._create_slider(len(boards))
        play_button = self._create_play_button(slider)
        pause_button = self._create_pause_button()
        fig = figure(x_range=(-1, self.n+1), y_range=(-1, self.n+1), width=600, height=600)

        # prepare transition frames
        frames = self._make_frames(boards, takahashi_pos_list, aoki_pos_list)

        # update source data based on slider value
        rect_source, takahashi_source, aoki_source = frames[0]
        callback = self._create_callback(frames, rect_source, takahashi_source, aoki_source)
        slider.js_on_change('value', callback)

        # Visualize the board states
        self.draw_vertical_walls(fig)
        self.draw_horizontal_walls(fig)
        self.draw_numbers(fig, rect_source)
        self.draw_positions(fig, takahashi_source, aoki_source)

        # Make layout of the visualization
        layout = column(slider, fig, row(play_button, pause_button))
        output_file("visualization.html")
        save(layout)

# Instantiate the Visualizer and visualize the board states
visualizer = Visualizer(n, WALL_V, WALL_H)
visualizer.visualize_with_slider(boards, takahashi_pos_list, aoki_pos_list)